In [1]:
from utils.setting import get_amplify_token
dotenv_path = "../.env"
amplify_token = get_amplify_token(dotenv_path)

Binaly変数 {0, 1}
イジング変数 {-1, 1}

In [9]:
from amplify import BinaryPoly, gen_symbols

In [4]:
#f = 2 * q_0 * q_1 + q_0 + q_1 - 1
f = BinaryPoly({(0, 1): 2, (0): 1, (1): 1, (): -1})
print(f)

2.000000 q_0 q_1 + q_0 + q_1 - 1.000000


In [6]:
g = BinaryPoly(f)
print(f)
print(g)

2.000000 q_0 q_1 + q_0 + q_1 - 1.000000
2.000000 q_0 q_1 + q_0 + q_1 - 1.000000


In [7]:
print(f + 1)
print(f + f)

2.000000 q_0 q_1 + q_0 + q_1
4.000000 q_0 q_1 + 2.000000 q_0 + 2.000000 q_1 - 2.000000


In [10]:
q = gen_symbols(BinaryPoly, 10)
f = 2 * q[0] * q[1] + q[0] + q[1] - 1

In [11]:
q_a = gen_symbols(BinaryPoly, 0, (2, 2))
q_b = gen_symbols(BinaryPoly, 4, (2, 2))
print(q_a)
print(q_b)

[[q_0, q_1], [q_2, q_3]]
[[q_4, q_5], [q_6, q_7]]


# 多項式の値を評価

In [12]:
q = gen_symbols(BinaryPoly, 10)
f = 2 * q[0] * q[1] + q[0] + q[1] - 1
print(f)
print(f.replace_all({0:1, 1:-1}))

2.000000 q_0 q_1 + q_0 + q_1 - 1.000000
-3.0


In [13]:
f.replace_all(lambda i: 2 * i + 1)

9.0

In [14]:
# Matrix

In [15]:
from amplify import BinaryMatrix

In [17]:
m = BinaryMatrix(3)
m[0, 0] = -2
m[0, 1] = 1
m[1, 2] = -1
m[2, 2] = 1

In [18]:
m.to_Poly()

q_0 q_1 - q_1 q_2 - 2.000000 q_0 + q_2

# 制約

In [20]:
from amplify import LogicPoly

# 両方1の時False, その他True
x = gen_symbols(LogicPoly, 2)
p = ~(x[0] & x[1])
print(p)

- x_0 x_1 + 1


In [21]:
from amplify.constraint import penalty

In [22]:
f = (~p).to_Poly()
print(f)
g = penalty(f)
print(g)

q_0 q_1
q_0 q_1 == 0


In [23]:
from amplify import (
    BinaryQuadraticModel,
    decode_solution,
    gen_symbols,
    LogicPoly,
    Solver,
)
from amplify.client import FixstarsClient

client = FixstarsClient()
client.token = amplify_token
client.parameters.timeout = 1000
client.parameters.outputs.duplicate = True
client.parameters.outputs.num_outputs = 0

model = BinaryQuadraticModel(g)
solver = Solver(client)
result = solver.solve(model)


In [25]:
for solution in result:
    values = solution.values
    print(decode_solution(x, values))

[0, 1]
[1, 0]
[0, 0]


# 制約条件

In [27]:
from amplify import BinaryPoly, BinaryQuadraticModel, gen_symbols, sum_poly
from amplify.constraint import equal_to

q = gen_symbols(BinaryPoly, 4)
f = 2 * q[0] * q[1] * q[2] + q[0] * q[1] + q[0] + q[1] + q[2] - 1
c1 = equal_to(sum_poly(q[:3]), 1)   # q_0 + q_1 + q_2 == 1
c2 = equal_to(sum_poly(q[1:]), 1)   # q_1 + q_2 + q_3 == 1
model = f + c1 + c2

print(model.input_poly)
print(model.input_constraints)

2.000000 q_0 q_1 q_2 + q_0 q_1 + q_0 + q_1 + q_2 - 1.000000
[(q_0 + q_1 + q_2 == 1.000000, 1), (q_1 + q_2 + q_3 == 1.000000, 1)]
